# Selenium 네이버 책 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (130 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

## 이번주 베스트셀러 긁어오기

In [ ]:
import time
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.implicitly_wait(3)
wd.execute_script('window.open("about:blank", "_blank");')
tabs = wd.window_handles
wd.switch_to.window(tabs[0])

wd.get("https://book.naver.com")

site_tab = wd.find_element_by_class_name('tab_cp_spt')
site_a = site_tab.find_elements_by_tag_name('a')
for i in range(len(site_a)): # book사이트별 반복
  try:
    wd.switch_to.window(tabs[0])
    site_tab = wd.find_element_by_class_name('tab_cp_spt')
    site_a = site_tab.find_elements_by_tag_name('a')[i]
    site_name = site_a.text
    print("[[", site_name, "]]")
    site_a.click()
    time.sleep(1)
    
    category_tab = wd.find_element_by_class_name('tab_cp_sub')
    category_a = category_tab.find_elements_by_tag_name('a')
    for j in range(len(category_a)): # 카테고리별 반복
      wd.switch_to.window(tabs[0])
      category_tab = wd.find_element_by_class_name('tab_cp_sub')
      category_a = category_tab.find_elements_by_tag_name('a')[j]
      category_name = category_a.find_element_by_tag_name('img').get_attribute('title')
      print("\t[", category_name, "]")
      category_a.click()
      time.sleep(1)

      title_list = []
      author_list = []
      publisher_list = []
      url_list = []
      date_list = []
      book_list = wd.find_elements_by_class_name('thumb_type')
      for k in range(len(book_list)): # 개별 book 반복
        wd.switch_to.window(tabs[0])
        book_list = wd.find_elements_by_class_name('thumb_type')
        book = book_list[k]
        book_url = book.find_element_by_tag_name('a').get_attribute('href')
        url_list.append(book_url)

        wd.switch_to.window(tabs[1]) # 새로운 탭에 book 링크 연결
        wd.get(book_url)

        info = wd.find_element_by_class_name('book_info')
        title = info.find_element_by_tag_name('a').text
        title_list.append(title)

        information = info.find_element_by_xpath('//*[@id="container"]/div[4]/div[1]/div[2]/div[2]').text.split("|")
        tmp_author_list = []    # 책 한권에 해당하는 저자 리스트
        for i in information:
            if '글' in i:
                tmp_author_list.append(i[2:])
            elif '그림' in i:
                tmp_author_list.append(i[3:])
            elif '저자' in i:
                tmp_author_list.append(i[3:])
        author_list.append(', '.join(tmp_author_list))
        publisher_list.append(information[-2])
        date_list.append(information[-1])

      bestseller_df = pd.DataFrame({"Title": title_list,
                                    "Author": author_list,
                                    "Publisher": publisher_list,
                                    "Date": date_list,
                                    "URL": url_list})
      print(bestseller_df)
  except:
    pass

[[ YES24 ]]
	[ 종합 ]
                                               Title  ...                                                URL
0              완전학습 바이블 배운 것을 100% 이해하는 후천적 공부머리의 비밀  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
1                       돈의 속성 최상위 부자가 말하는 돈에 대한 모든 것  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
2           이토록 공부가 재미있어지는 순간 공부에 지친 청소년들을 위한 힐링 에세이  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
3                       검찰개혁과 촛불시민 조국 사태로 본 정치검찰과 언론  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
4                     설민석의 세계사 대모험 6 미국 편 : 인디펜던스 데이  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
5                   살고 싶다는 농담(양장본 HardCover) 허지웅 에세이  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
6                   김미경의 리부트 코로나로 멈춘 나를 다시 일으켜 세우는 법  ...  http://book.naver.com/bookdb/book_detail.nhn?b...
7                                        바젤3 모멘트 개정판  ...  http://book.naver.com/bookdb/book_detail.nhn?b...